# Luokittelu

JAMK examples #10: 10_ml_revenue_classification

Ladataan käytettävät paketit.

In [1]:

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

Luetaan käytettävä aineisto:

In [2]:
csvUrl = "https://raw.githubusercontent.com/RaimoHaikari/paikkatieto/master/misc/sales_data.csv"
df = pd.read_csv(csvUrl)

df.shape

(1000, 6)

Seuraavaksi:

- tiputetaan NA-arvoja sisältävät tietueet
- muutetaan date-sarake aikaleimaksi
- lasketaan päivittäinen liikevaihto
- muunnetaan aineisto aikasarjaksi

In [3]:
# Handle missing values
# df = df.dropna()
df = df.dropna().reset_index()

# Convert the 'date' column to datetime
df['date'] = pd.to_datetime(df['date'])

# Calculate the daily revenue
df['revenue'] = df['sales'] * df['price']

df.set_index('date', inplace=True)

## TÄHÄN JÄÄTIIN....